In [1]:
import pandas as pd
from decimal import Decimal

In [27]:
# read all values as a string
df = pd.read_csv("/Users/pedrosanhueza/EXOXY/Administrative Documents/Jobs/geoaire/Planilla.csv", dtype=str)

In [28]:
# replace comma with dots
df = df.applymap(lambda x: str(x).replace(',', '.'))

In [29]:
# convert values to decimals
df = df.applymap(lambda x: Decimal(x) if pd.notna(x) else x)

In [30]:
lai = Decimal('0.15')
tc = Decimal('0.05')

In [31]:
df['Velocidad del viento (m/s)'] = df['Velocidad del viento (m/s)'].apply(lambda x: round(x,0))

table1 = {
    'Velocidad del viento (m/s)': ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13'],
    'Promedio': ['0', '0.03', '0.09', '0.15', '0.17', '0.19', '0.2', '0.56', '0.92', '0.92', '2.11', '2.11', '2.11', '2.11'],
    'Mínimo': ['0', '0.006', '0.012', '0.018', '0.022', '0.025', '0.029', '0.056', '0.082', '0.082', '0.57', '0.57', '0.57', '0.57'],
    'Máximo': ['0', '0.042', '0.163', '0.285', '0.349', '0.414', '0.478', '1.506', '2.534', '2.534', '2.534', '2.534', '2.534', '2.534'],
    '% Resuspensión': ['0', '1.5', '3', '4.5', '6', '7.5', '9', '10', '11', '12', '13', '16', '20', '23']
}

reference_table = pd.DataFrame(table1).applymap(lambda x: Decimal(x) if pd.notna(x) else x)

In [32]:
df = pd.merge(df, reference_table, on='Velocidad del viento (m/s)', how='left')

df['Vd (cm/s)']     = lai * df['Promedio']
df['Vd,min (cm/s)'] = lai * df['Mínimo']
df['Vd,max (cm/s)'] = lai * df['Máximo']

In [41]:
flux = pd.DataFrame()
flux['MP2,5 (g/m³)']    = df['MP2,5 (µg/m³)'] / 1000 / 1000
flux["Vd,min (m/s)"]    = df['Vd,min (cm/s)'] / 100
flux["Vd,max (m/s)"]    = df['Vd,max (cm/s)'] / 100
flux["Vd (m/s)"]        = df['Vd (cm/s)'] / 100
flux["fmin,t (g/m2*h)"] = flux['MP2,5 (g/m³)'] * flux["Vd,min (m/s)"] * 3600
flux["fmax,t (g/m2*h)"] = flux['MP2,5 (g/m³)'] * flux["Vd,max (m/s)"] * 3600
flux["ft (g/m2*h)"]     = flux['MP2,5 (g/m³)'] * flux["Vd (m/s)"] * 3600

In [42]:
flux

,"MP2,5 (g/m³)","Vd,min (m/s)","Vd,max (m/s)",Vd (m/s),"fmin,t (g/m2*h)","fmax,t (g/m2*h)",ft (g/m2*h)
0,0.0000256,0.000027,0.0004275,0.000225,0.0000024883200,0.00003939840000,0.0000207360000
1,0.000007,0.000027,0.0004275,0.000225,6.80400E-7,0.0000107730000,0.000005670000
2,0.0000054,0.000018,0.0002445,0.000135,3.499200E-7,0.00000475308000,0.0000026244000
3,0.000004,0.000009,0.000063,0.000045,1.29600E-7,9.07200E-7,6.48000E-7
4,0.0000115,0.000009,0.000063,0.000045,3.726000E-7,0.0000026082000,0.0000018630000
...,...,...,...,...,...,...,...
8317,0.0000101,0.000033,0.0005235,0.000255,0.0000011998800,0.00001903446000,0.0000092718000
8318,0.0000118,0.000027,0.0004275,0.000225,0.0000011469600,0.00001816020000,0.0000095580000
8319,0.0000125,0.000027,0.0004275,0.000225,0.0000012150000,0.00001923750000,0.0000101250000
8320,0.0000108,0.000018,0.0002445,0.000135,6.998400E-7,0.00000950616000,0.0000052488000


In [49]:
resuspension = pd.DataFrame()
resuspension['% Resuspensión']  = df['% Resuspensión']
resuspension['fmin,t (g/m2*h)'] = flux["fmin,t (g/m2*h)"]
resuspension['fmax,t (g/m2*h)'] = flux["fmax,t (g/m2*h)"]
resuspension['ft (g/m2*h)']     = flux["ft (g/m2*h)"]
resuspension['At min (g/m2*h)'] = Decimal('0')
resuspension['At max (g/m2*h)'] = Decimal('0')
resuspension['At (g/m2*h)']     = Decimal('0')
resuspension['Rmin (g/m2*h)']   = Decimal('0')
resuspension['Rmax (g/m2*h)']   = Decimal('0')
resuspension['Rt (g/m2*h)']     = Decimal('0')

resuspension_zip = zip(
    df['% Resuspensión'],
    flux["fmin,t (g/m2*h)"],
    flux["fmax,t (g/m2*h)"],
    flux["ft (g/m2*h)"]
)

# for res, fmin, fmax, ft in resuspension_zip:


4.5 0.0000024883200 0.00003939840000 0.0000207360000
4.5 6.80400E-7 0.0000107730000 0.000005670000
3 3.499200E-7 0.00000475308000 0.0000026244000
1.5 1.29600E-7 9.07200E-7 6.48000E-7
1.5 3.726000E-7 0.0000026082000 0.0000018630000
1.5 3.855600E-7 0.0000026989200 0.0000019278000
0 0E-9 0E-9 0E-9
1.5 4.795200E-7 0.0000033566400 0.0000023976000
3 0.0000013219200 0.00001795608000 0.0000099144000
4.5 0.0000024202800 0.00003832110000 0.0000201690000
6 0.0000017344800 0.00002751516000 0.0000134028000
6 0.0000012355200 0.00001959984000 0.0000095472000
6 8.197200E-7 0.00001300374000 0.0000063342000
6 5.108400E-7 0.00000810378000 0.0000039474000
6 4.870800E-7 0.00000772686000 0.0000037638000
7.5 7.4250000E-7 0.0000122958000 0.0000056430000
7.5 8.100000E-7 0.000013413600 0.000006156000
7.5 6.750000E-7 0.000011178000 0.000005130000
6 8.672400E-7 0.00001375758000 0.0000067014000
6 6.534000E-7 0.00001036530000 0.0000050490000
6 0.0000010454400 0.00001658448000 0.0000080784000
4.5 7.873200E-7 0.00001

In [39]:
# running time: 180 seconds

resuspension = pd.DataFrame()
resuspension['% Resuspensión']  = df['% Resuspensión']
resuspension['fmin,t (g/m2*h)'] = flux["fmin,t (g/m2*h)"]
resuspension['fmax,t (g/m2*h)'] = flux["fmax,t (g/m2*h)"]
resuspension['ft (g/m2*h)']     = flux["ft (g/m2*h)"]
resuspension['At min (g/m2*h)'] = Decimal('0')
resuspension['At max (g/m2*h)'] = Decimal('0')
resuspension['At (g/m2*h)']     = Decimal('0')
resuspension['Rmin (g/m2*h)']   = Decimal('0')
resuspension['Rmax (g/m2*h)']   = Decimal('0')
resuspension['Rt (g/m2*h)']     = Decimal('0')

rows = []
for fila in range(resuspension.shape[0]-1):
# for fila in range(2):

    resuspension['Rmin (g/m2*h)']   = ( resuspension['At min (g/m2*h)'].shift(1) + resuspension['fmin,t (g/m2*h)'] ) * resuspension['% Resuspensión'] / Decimal('100')
    resuspension['At min (g/m2*h)'] = resuspension['At min (g/m2*h)'].shift(1) + resuspension['fmin,t (g/m2*h)'] - resuspension['Rmin (g/m2*h)']
    resuspension['Rmax (g/m2*h)']   = ( resuspension['At max (g/m2*h)'].shift(1) + resuspension['fmax,t (g/m2*h)'] ) * resuspension['% Resuspensión'] / Decimal('100')
    resuspension['At max (g/m2*h)'] = resuspension['At max (g/m2*h)'].shift(1) + resuspension['fmax,t (g/m2*h)'] - resuspension['Rmax (g/m2*h)']
    resuspension['Rt (g/m2*h)']     = ( resuspension['At (g/m2*h)'].shift(1) + resuspension['ft (g/m2*h)'] ) * resuspension['% Resuspensión'] / Decimal('100')
    resuspension['At (g/m2*h)']     = resuspension['At (g/m2*h)'].shift(1) + resuspension['ft (g/m2*h)'] - resuspension['Rt (g/m2*h)']

    row = {}
    row['Rmin (g/m2*h)']   = resuspension['Rmin (g/m2*h)'].iloc[fila+1]
    row['At min (g/m2*h)'] = resuspension['At min (g/m2*h)'].iloc[fila+1]
    row['Rmax (g/m2*h)']   = resuspension['Rmax (g/m2*h)'].iloc[fila+1]
    row['At max (g/m2*h)'] = resuspension['At max (g/m2*h)'].iloc[fila+1]
    row['Rt (g/m2*h)']     = resuspension['Rt (g/m2*h)'].iloc[fila+1]
    row['At (g/m2*h)']     = resuspension['At (g/m2*h)'].iloc[fila+1]

    rows.append(row)


In [ ]:
pd.set_option('display.float_format', lambda x: '%.16f' % x)

In [ ]:
resuspension2 = pd.DataFrame(rows)

flux_neto = pd.DataFrame()

flux_neto['Fmin (g/m2*h)'] = flux['fmin,t (g/m2*h)'] - resuspension2['Rmin (g/m2*h)']
flux_neto['Fmax(g/m2*h)']  = flux['fmax,t (g/m2*h)'] - resuspension2['Rmax (g/m2*h)']
flux_neto['Ft (g/m2*h)']   = flux['ft (g/m2*h)']     - resuspension2['Rt (g/m2*h)']

flux_neto

In [ ]:
cambio_concentracion = pd.DataFrame()

cambio_concentracion['C (ug/m3*h)']         = df['MP2,5 (µg/m³)']
cambio_concentracion['H (m)']               = df['Altura de Mezcla (m)']
cambio_concentracion['F (ug/m2*h)']         = flux_neto['Ft (g/m2*h)'] * 1000 * 1000
cambio_concentracion['Mtotal (ug/m2*h)']    = cambio_concentracion['C (ug/m3*h)'] * cambio_concentracion['H (m)']
cambio_concentracion['Iunit (%)']           = cambio_concentracion.apply(lambda row:
    (row['F (ug/m2*h)'] * 100 / row['Mtotal (ug/m2*h)'])
    if row['F (ug/m2*h)'] > 0
    else row['F (ug/m2*h)'] * 100 / (row['Mtotal (ug/m2*h)'] + row['F (ug/m2*h)']),
    axis=1)
cambio_concentracion['Itotal (%)']          = (cambio_concentracion['F (ug/m2*h)'] * tc) / (cambio_concentracion['F (ug/m2*h)'] * tc / 100 + cambio_concentracion['Mtotal (ug/m2*h)'])
cambio_concentracion['ΔC (ug/m3*h)']        = cambio_concentracion['C (ug/m3*h)'] / (1 - cambio_concentracion['Itotal (%)'] / 100) - cambio_concentracion['C (ug/m3*h)']

cambio_concentracion

In [ ]:
cambio_concentracion['ΔC (ug/m3*h)'].sum()

In [ ]:
user_input = 3
A = [5,4,4,2,3]
B = [3,1,1,5,3]

# Initialize the sum
C = user_input
D = 0
result = 0

# Calculate the sum dynamically based on the pattern
for a,b in zip(A,B):
    D = C * b + a
    print(f'{D} = {C} * {b} + {a}')
    C = C + D
    print(C)
    result += D

print(result)


In [ ]:
resuspension

In [ ]:
columns = zip(
    resuspension['% Resuspensión'][1:],
    resuspension['fmin,t (g/m2*h)'][1:]
)

In [ ]:
At_min = 0

for Resuspensión,fmin in columns:
    Rmin = ( At_min + fmin ) * Resuspensión / 100
    print(f'Rmin: {Rmin} = ({At_min} + {fmin}) * {Resuspensión} / 100')
    At_min = At_min + fmin - Rmin
    print(At_min, "\n")

print(At_min, Rmin)


In [ ]:
a = 0.0001
b = 0.0002

print(a,b)

In [ ]:
a * b

In [ ]:
a = 0.0001
b = 0.0002

a *= 1000
b *= 1000

print(a,b)

In [ ]:
(a * 1000 * b * 1000) / 1000 / 1000

In [ ]:
a*b

In [ ]:
from decimal import Decimal, getcontext

getcontext().prec = 20

a = Decimal('0.1')
b = Decimal('0.2')

a*b